In [1]:
from dcicutils import ff_utils

# STORY
# Get all awsem wfrs without a contributing lab, look at their input files
# This scipt consider two cases, 1) fastq input files, 2) processed input files
# If fastq is the input file, lab of that file is take and used as the lab
# if processed file is the input, first check the lab, if it is dcic take the 
# contributing lab as the lab.
# Add this lab to the contributing lab of the wfr, and if wfr has a qc object,
# add lab to its contributing lab

my_env = 'data'
my_auth = ff_utils.get_authentication_with_server({}, ff_env = my_env)

# find all runs from 4dn lab without a contributing lab
q = "/search/?lab.uuid=828cd4fe-ebb0-4b36-a94a-d2e3a36cc989" + \
    "&run_status=complete&type=WorkflowRunAwsem&contributing_labs.uuid=No+value" + \
    "&limit=all"

all_awsem_resp = ff_utils.search_metadata(q, key=my_auth)
print(len(all_awsem_resp))

2619


In [3]:
counter = 0
for a_resp in all_awsem_resp: 
    counter += 1
    if counter % 100 == 0:
        print(counter)
    # find the identity
    input_files = a_resp['input_files']
    my_lab = ''
    for an_input in input_files:
        # if this wfr ran on processed files
        input_id = an_input['value']['@id']
        if input_id.startswith('/files-processed/'):
            input_resp = ff_utils.get_metadata(an_input['value']['@id'], key=my_auth)
            my_lab = input_resp['lab']['@id']
            if my_lab == '/labs/4dn-dcic-lab/':
                try:
                    my_lab = input_resp['contributing_labs'][0]['@id']
                except:
                    my_lab = 'no contrbuting lab'
            break
        # if this wfr ran on raw files
        elif input_id.startswith('/files-fastq/') or input_id.startswith('/files-microscopy/'):
            input_resp = ff_utils.get_metadata(input_id, key=my_auth)
            my_lab = input_resp['lab']['@id']
            break
            
    if my_lab == '':
        # some submitted files that gone through md5 and other weird ones
        print(a_resp['uuid'], 'did not have a valid input')
        continue
    
    if my_lab == 'no contrbuting lab':
        # ignore some old wfrs
        print(a_resp['uuid'], 'did not have input with contr lab')
        continue
 
    if my_lab == '/labs/4dn-dcic-lab/':
        print(a_resp['uuid'],'is from 4dn')
        continue
    
    # add contributing lab to wfr
    ff_utils.patch_metadata({'contributing_labs':[my_lab]},obj_id=a_resp['uuid'], key=my_auth)
    
    # check if there are any qcs from the wfr
    qcs = a_resp.get('output_quality_metrics')
    if qcs:
        qc_uuid = qcs[0]['value']['uuid']
        ff_utils.patch_metadata({'contributing_labs':[my_lab]},obj_id=qc_uuid, key=my_auth)


c33e7d69-62d0-4ce9-bacf-b535bfba07f9 did not have a valid input
9d458e49-0325-4176-b1c8-1da11d37bf22 did not have a valid input
a5cf9f0f-6b4f-4a0a-9998-0b7e6cfce5c9 did not have a valid input
275ffddd-3917-4c74-87d7-879ad2f8d4e0 did not have a valid input
77a8692a-9abd-4201-9d2a-70af6cf8e1db did not have a valid input
897b9325-730f-4e58-a7e7-3321fa454f1f did not have a valid input
6e15c57d-54b6-4fcf-9fb0-a14c067b67d6 did not have a valid input
d71f7583-e59e-456c-afb7-285711bd091d did not have a valid input
cf4cfa6b-5bbc-4538-985f-bb2dfefd187f did not have input with contr lab
d5977093-ffea-4303-9687-8fcc43de113e did not have a valid input
9a5bbc6f-23de-4093-b3b1-e45b3d5ac697 is from 4dn
100
200
300
400
500
600
700
800
b07827e3-3a0a-4b2b-bf69-1ca988b46fd1 is from 4dn
9b8c14f8-20ba-4b4d-af89-150a34a0d98d did not have a valid input
bc3971ff-7a28-462a-a30d-75ef5b55b352 did not have a valid input
3f320894-15f0-41a5-bb12-b2fa55939620 did not have a valid input
581e0da2-e534-4799-bcf9-1551909